# Qual professor reprova mais e qual aprova mais ?

In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, inspect 

In [3]:
engine = create_engine('sqlite:///:memory:') # Criando banco SQL
engine 

Engine(sqlite:///:memory:)

In [4]:
tabela_notas = pd.read_excel('files/tabela_nota.xlsx') # Importando tabela de notas
tabela_notas.drop(tabela_notas.loc[tabela_notas['id_materia']==0].index, inplace=True) # Não é possivel saber qual materia ele fez
tabela_notas['id'] = range(tabela_notas.shape[0])
tabela_notas.set_index('id', inplace=True)
tabela_notas.index = tabela_notas.index + 1
tabela_notas.rename(columns={'id_professor(a)': 'id_professor'}, inplace=True)
tabela_notas.to_sql('notas', engine) 
inspector = inspect(engine) 

In [67]:
tabela_professores = pd.read_excel('files/tabela_professor.xlsx', index_col=False) # Importando tabela de professores
tabela_professores.set_index('Unnamed: 0', inplace=True)
tabela_professores.index.rename('id', inplace=True)

In [6]:
# Para saber qual professor reprova mais tem que pegar todas as notas que ele deu e ver o tanto de notas vermelhas que ele deu, notas_vermelhas/totas_notas

In [66]:
tabela_materias = pd.read_excel('files/tabela_materia.xlsx', index_col=False) # Importando tabela de professores
tabela_materias.set_index('Unnamed: 0', inplace=True)
tabela_materias.index.rename('id', inplace=True)

In [64]:
# Funções

# Selecionando professor para fazer a busca da tabela de todas as avaliações realizadas por ele
def seleciona_professor(nome: str):
    professor = tabela_professores[tabela_professores['nome'] == nome].index
    professor = professor[0]
    return professor

# Funcao que busca a tabela de avaliacoes feitas pelo professor selecionado
def busca_tabela_avaliacoes_professor(nome: str):
    query = f'select * from notas where id_professor = {seleciona_professor(nome)}'
    notas_professor = pd.read_sql(query, engine)
    notas_professor.drop('id', axis=1, inplace=True) # Deletando a coluna de id
    return notas_professor

# Funcao que pega o dataframe de notas do professor e transforma
# em outro dataframe com 2 dados somente ['id_materia'] e ['nota'], 
# que são as colunaas que irei usar.
def simplifica_tabela_avaliacoes(nome: str):
    notas_professor = busca_tabela_avaliacoes_professor(nome)
    notas_profesor_simplificada = notas_professor[['id_materia', 'nota']]
    return notas_profesor_simplificada

# Funcao que calcula quantidade de notas dadas por cada professor
def qtd_notas_avaliadas(notas_professor_simplificada):
    qtd_notas = notas_professor_simplificada.shape[0]
    return qtd_notas

# Funcao que calcula a quantidade de notas vermelhas que cada professor deu de acordo com cada nota_minima de cada materia
def qtd_notas_avaliadas_vermelhas(qtd_notas_avaliadas: int, notas_professor_simplificada):
    qtd_notas_vermelhas = 0
    
    for i in range(qtd_notas_avaliadas):
        select = notas_professor_simplificada['id_materia'][i]
        if tabela_materias['nota_minima'][tabela_materias.index[select-1]] > notas_professor_simplificada['nota'][i]:
            qtd_notas_vermelhas += 1
    
    return qtd_notas_vermelhas

# Pega o nome do professor selecionado
def traz_nome_professor(id_professor: int) -> str:
    nome_professor = tabela_professores['nome'][id_professor]
    return nome_professor

# Funcao que calcula a taxa de reprovação dos alunos para cada professor
def taxa_reprovacao(nome: str):
    notas_professor = simplifica_tabela_avaliacoes(nome)
    qtd_notas = qtd_notas_avaliadas(notas_professor)
    qtd_notas_vermelhas = qtd_notas_avaliadas_vermelhas(qtd_notas, notas_professor)
    
    taxa_reprovacao_professor = (qtd_notas_vermelhas / qtd_notas) * 100 # Calculo taxa de reprovação
    return taxa_reprovacao_professor

# Exportar no excel
def exporta_para_excel(itens: list, nome_coluna_itens: str, valores: list, nome_coluna_valores: str, caminho: str):
    dados = pd.DataFrame({nome_coluna_itens: itens, nome_coluna_valores: valores})
    dados.to_excel(caminho, index=False)

## Calculando taxa de reprovação para todos os professores

In [65]:
taxa_reprovacao_professores = []
qtd_professores = tabela_professores.shape[0] + 1

for i in range(1, qtd_professores):
    taxa_reprovacao_professores.append(taxa_reprovacao(traz_nome_professor(i)))

[62.5, 63.63636363636363, 58.333333333333336, 62.5, 77.77777777777779]

In [68]:
professores = list(tabela_professores['nome'])

In [45]:
exporta_para_excel(professores, 'Professor', taxa_reprovacao_professores, 'Taxa de reprovação', 'files/files_questions/question_3_4/taxa_reprovacao_por_prof.xlsx')